In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Q1

In [15]:
# a

iris.groupby('species').agg(['describe'])


sepal_length                                              \
               describe                                               
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
              describe         ...     describe         describe          
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

In [12]:
# b

s = iris[iris.species == 'setosa']['sepal_width']
q1, q3 = s.quantile(0.25), s.quantile(0.75)
iqr = q3 - q1
q1, q3, iqr

(3.2, 3.6750000000000003, 0.4750000000000001)

In [13]:
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr
s[(s < lower) | (s > upper)] = np.nan
s.isna().sum()

2

In [14]:
s.mean(), s.std()

(3.4312500000000004, 0.32034306743094015)

In [19]:
def get_new_stat(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    s[(s < lower) | (s > upper)] = np.nan
    return s.isna().sum(), s.mean(), s.std()

In [17]:
iris.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [18]:
iris.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

s = iris[iris.species == 'setosa']['sepal_width']

In [20]:
for species in iris.species.unique():
    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        outlier, new_mean, new_std = get_new_stat(s)
        print(outlier, new_mean, new_std)


0 5.005999999999999 0.3524896872134512
2 3.4312500000000004 0.32034306743094015
4 1.4608695652173913 0.12906201931985212
2 0.23333333333333328 0.08588298157855664
0 5.936 0.5161711470638635
0 2.7700000000000005 0.3137983233784114
1 4.285714285714285 0.4377975178854565
0 1.3259999999999998 0.197752680004544
1 6.6224489795918355 0.593459246748267
3 2.95531914893617 0.26027452951125174
0 5.552 0.5518946956639835
0 2.026 0.27465005563666733


In [24]:
all = []
for species in iris.species.unique():
    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        old_mean, old_std = round(s.mean(),3), round(s.std(),4)
        outlier, new_mean, new_std = get_new_stat(s)
        d = {'species':species, 'feature':feature, 'old_mean':old_mean, 'old_std':old_std, 'outlier갯수':outlier, 'newmean':round(new_mean,3), 'newstd':round(new_std,4)}
        all.append(d)
df = pd.DataFrame(all)
df.set_index(['species', 'feature'], inplace=True)
df


old_mean  old_std  outlier갯수  newmean  newstd
species    feature                                                    
setosa     sepal_length     5.006   0.3525          0    5.006  0.3525
           sepal_width      3.428   0.3791          2    3.431  0.3203
           petal_length     1.462   0.1737          4    1.461  0.1291
           petal_width      0.246   0.1054          2    0.233  0.0859
versicolor sepal_length     5.936   0.5162          0    5.936  0.5162
           sepal_width      2.770   0.3138          0    2.770  0.3138
           petal_length     4.260   0.4699          1    4.286  0.4378
           petal_width      1.326   0.1978          0    1.326  0.1978
virginica  sepal_length     6.588   0.6359          1    6.622  0.5935
           sepal_width      2.974   0.3225          3    2.955  0.2603
           petal_length     5.552   0.5519          0    5.552  0.5519
           petal_width      2.026   0.2747          0    2.026  0.2747

Q2

In [5]:
titanic = sns.load_dataset('titanic')
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


In [26]:
# a

titanic['category1'] = titanic.apply(lambda x: x.sex if x.age>=20 else 'child', axis=1)
titanic.tail()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [28]:
# b
titanic['age'].fillna(titanic.age.mean(), inplace=True)
titanic['category1'] = titanic.apply(lambda x: x.sex if x.age>=20 else 'child', axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [30]:
#c  survived의 평균 = 생존률
titanic.groupby(['sex', 'class', 'embark_town'])[['survived']].mean()

survived
sex    class  embark_town          
female First  Cherbourg    0.976744
              Queenstown   1.000000
              Southampton  0.958333
       Second Cherbourg    1.000000
              Queenstown   1.000000
              Southampton  0.910448
       Third  Cherbourg    0.652174
              Queenstown   0.727273
              Southampton  0.375000
male   First  Cherbourg    0.404762
              Queenstown   0.000000
              Southampton  0.354430
       Second Cherbourg    0.200000
              Queenstown   0.000000
              Southampton  0.154639
       Third  Cherbourg    0.232558
              Queenstown   0.076923
              Southampton  0.128302

In [29]:
#d

titanic['age']
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

age2 = pd.cut(titanic.age, bins, labels=labels)
titanic['age2'] = age2
titanic.groupby(['age2'])[['survived']].mean()


,survived
age2,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


In [ ]:
#e



Q3

In [31]:
MPG = sns.load_dataset('mpg')
MPG.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [32]:
#a

MPG['hp_per_cc'] = MPG.horsepower / MPG.displacement
MPG.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576


In [54]:
list(MPG['name'])

['chevrolet chevelle malibu',
 'buick skylark 320',
 'plymouth satellite',
 'amc rebel sst',
 'ford torino',
 'ford galaxie 500',
 'chevrolet impala',
 'plymouth fury iii',
 'pontiac catalina',
 'amc ambassador dpl',
 'dodge challenger se',
 "plymouth 'cuda 340",
 'chevrolet monte carlo',
 'buick estate wagon (sw)',
 'toyota corona mark ii',
 'plymouth duster',
 'amc hornet',
 'ford maverick',
 'datsun pl510',
 'volkswagen 1131 deluxe sedan',
 'peugeot 504',
 'audi 100 ls',
 'saab 99e',
 'bmw 2002',
 'amc gremlin',
 'ford f250',
 'chevy c20',
 'dodge d200',
 'hi 1200d',
 'datsun pl510',
 'chevrolet vega 2300',
 'toyota corona',
 'ford pinto',
 'amc gremlin',
 'plymouth satellite custom',
 'chevrolet chevelle malibu',
 'ford torino 500',
 'amc matador',
 'chevrolet impala',
 'pontiac catalina brougham',
 'ford galaxie 500',
 'plymouth fury iii',
 'dodge monaco (sw)',
 'ford country squire (sw)',
 'pontiac safari (sw)',
 'amc hornet sportabout (sw)',
 'chevrolet vega (sw)',
 'pontiac fir

In [63]:
#b

MPG['manu'] = MPG.name.apply(lambda x: x.split()[0])
MPG['model'] = MPG.name.apply(lambda x: ' '.join(x.split()[1:]))
MPG.drop(columns=['name'], inplace=True)
MPG.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manu,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [65]:
#c
MPG.groupby('cylinders')['mpg'].mean()

cylinders
3    20.550000
4    29.286765
5    27.366667
6    19.985714
8    14.963107
Name: mpg, dtype: float64

In [66]:
#d
MPG.groupby('origin')['hp_per_cc'].mean()

origin
europe    0.740005
japan     0.789485
usa       0.512063
Name: hp_per_cc, dtype: float64

In [67]:
#e
MPG.manu.value_counts()

ford             51
chevrolet        43
plymouth         31
amc              28
dodge            28
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
mazda            10
oldsmobile       10
peugeot           8
fiat              8
audi              7
volvo             6
vw                6
chrysler          6
renault           5
saab              4
subaru            4
opel              4
chevy             3
cadillac          2
maxda             2
bmw               2
mercedes-benz     2
nissan            1
vokswagen         1
mercedes          1
hi                1
triumph           1
toyouta           1
capri             1
chevroelt         1
Name: manu, dtype: int64

In [68]:
s = MPG.manu.value_counts()
s[s >= 5]

ford          51
chevrolet     43
plymouth      31
amc           28
dodge         28
toyota        25
datsun        23
buick         17
pontiac       16
volkswagen    15
honda         13
mercury       11
mazda         10
oldsmobile    10
peugeot        8
fiat           8
audi           7
volvo          6
vw             6
chrysler       6
renault        5
Name: manu, dtype: int64

In [70]:
big = s[s >= 5].index
big

Index(['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun',
       'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda',
       'oldsmobile', 'peugeot', 'fiat', 'audi', 'volvo', 'vw', 'chrysler',
       'renault'],
      dtype='object')

In [71]:
'ford' in big

True

In [73]:
MPG.manu.isin(big).tail()

393     True
394     True
395     True
396     True
397    False
Name: manu, dtype: bool

In [75]:
big_mpg = MPG[MPG.manu.isin(big)]
big_mpg.groupby('manu')['mpg'].mean().sort_values(ascending=False).head(5)

manu
vw         39.016667
honda      33.761538
renault    32.880000
datsun     31.113043
mazda      30.860000
Name: mpg, dtype: float64